# Классификация на эмбеддингах

In [5]:
import numpy as np
import pandas as pd
import torch
import transformers
from tqdm import notebook
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split

import re 

In [2]:
df_tweets = pd.read_csv('/datasets/tweets.csv')

In [3]:
tokenizer = transformers.BertTokenizer(
    vocab_file='/datasets/ds_bert/vocab.txt')

tokenized = df_tweets['text'].apply(
    lambda x: tokenizer.encode(x, add_special_tokens=True))

max_len = 0
for i in tokenized.values:
    if len(i) > max_len:
        max_len = len(i)

padded = np.array([i + [0]*(max_len - len(i)) for i in tokenized.values])

attention_mask = np.where(padded != 0, 1, 0)

In [13]:
padded

array([[  101,   168, 10934, ...,     0,     0,     0],
       [  101,   886,   128, ...,     0,     0,     0],
       [  101, 88488,   168, ...,     0,     0,     0],
       ...,
       [  101,  1097,  1064, ...,     0,     0,     0],
       [  101,  1086,  1965, ...,     0,     0,     0],
       [  101,   168, 11061, ...,     0,     0,     0]])

In [8]:
config = transformers.BertConfig.from_json_file(
    '/datasets/ds_bert/bert_config.json')
model = transformers.BertModel.from_pretrained(
    '/datasets/ds_bert/rubert_model.bin', config=config)

Some weights of the model checkpoint at /datasets/ds_bert/rubert_model.bin were not used when initializing BertModel: ['cls.predictions.transform.dense.weight', 'cls.predictions.decoder.bias', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [11]:
batch_size = 100
embeddings = []
for i in notebook.tqdm(range(padded.shape[0] // batch_size)):
        batch = torch.LongTensor(padded[batch_size*i:batch_size*(i+1)]) 
        attention_mask_batch = torch.LongTensor(attention_mask[batch_size*i:batch_size*(i+1)])
        
        with torch.no_grad():
            batch_embeddings = model(batch, attention_mask=attention_mask_batch)
        
        embeddings.append(batch_embeddings[0][:,0,:].numpy())

  0%|          | 0/50 [00:00<?, ?it/s]

In [12]:
features = np.concatenate(embeddings)
features

# обучите и протестируйте модель
# < напишите код здесь >

array([[-0.27149776, -0.5590756 , -0.28098318, ..., -0.9203016 ,
         1.2013047 ,  0.01502726],
       [-0.3274793 ,  0.41710758, -0.82698894, ..., -0.9875183 ,
         0.43697062, -0.45578933],
       [-0.17064211, -0.06034701, -0.35421452, ...,  0.00830599,
         0.14736848, -0.35230753],
       ...,
       [ 0.03029836, -0.85618323, -0.6820553 , ..., -0.3185835 ,
         0.5538569 , -0.9658166 ],
       [-0.08716808,  0.02353097, -0.3716771 , ...,  0.47204947,
        -0.01140365, -0.40209013],
       [ 0.15282145,  0.4337314 , -1.826283  , ..., -1.0342153 ,
        -0.06139715, -1.4942064 ]], dtype=float32)

In [31]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [18]:
df_tweets['positive'].shape
features.shape

(5000, 768)

In [28]:
x_train, x_test, y_train, y_test =  train_test_split(features,df_tweets['positive'], test_size = 0.5)

In [29]:
lr = LogisticRegression()
lr.fit(x_train, y_train)

/opt/conda/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

In [33]:
accuracy_score(lr.predict(x_train), y_train)

0.9996